In [1]:
import pandas as pd
import numpy
from palmerpenguins import load_penguins
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import label_binarize

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
X = train.drop(['Transported'], axis = 1)
y = train['Transported']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25)

In [6]:
 ('dummify_ecg', make_pipeline(cat_imputer, enc1), ['HomePlanet']),
    ('dummify_cp', make_pipeline(cat_imputer, enc1), ['CryoSleep']),
    ("dummify_dest", make_pipeline(cat_imputer, enc1), ['Destination']),
    ("dummify_vip", make_pipeline(cat_imputer, enc1), ['VIP']),
    ('nums', make_pipeline(num_imputer, FunctionTransformer(lambda x: x)), 
     ['Age', "RoomService", "FoodCourt", "ShoppingMall","Spa", "VRDeck"])

IndentationError: unexpected indent (1418156183.py, line 2)

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
enc1 = OneHotEncoder(handle_unknown='ignore')

# Separate imputers for categorical and numerical data
cat_imputer = SimpleImputer(strategy='most_frequent')
num_imputer = SimpleImputer(strategy='mean')

ct = ColumnTransformer([
    ('dummify_ecg', make_pipeline(cat_imputer, enc1), ['HomePlanet']),
    ('dummify_cp', make_pipeline(cat_imputer, enc1), ['CryoSleep']),
    ("dummify_dest", make_pipeline(cat_imputer, enc1), ['Destination']),
    ("dummify_vip", make_pipeline(cat_imputer, enc1), ['VIP']),
    ('nums', make_pipeline(num_imputer, StandardScaler()), 
     ['Age', "RoomService", "FoodCourt", "ShoppingMall","Spa", "VRDeck"])
], remainder='drop')

# Pipeline with KNN Classifier
Knn1 = Pipeline([
    ('dummify_everything', ct),
    ('KNN', KNeighborsClassifier())
])

# Grid Search
neigh = {'KNN__n_neighbors': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]}
gscv1 = GridSearchCV(Knn1, neigh, cv=5, scoring='roc_auc')
gscv1fit = gscv1.fit(X_train, y_train)

# Ensure that the best parameters are found
best_n_neighbors = gscv1fit.best_params_.get('KNN__n_neighbors', 5)  # Default to 5 if not found

# Final KNN pipeline
Knn1 = Pipeline([
    ('dummify_everything', ct),
    ('KNN', KNeighborsClassifier(n_neighbors=best_n_neighbors))
])

fit = Knn1.fit(X_train, y_train)
cross_val_score(Knn1,X,y, scoring = 'accuracy').mean()

0.7831608975966639

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
enc1 = OneHotEncoder(handle_unknown='ignore')

# Separate imputers for categorical and numerical data
cat_imputer = SimpleImputer(strategy='most_frequent')
num_imputer = SimpleImputer(strategy='mean')

ct = ColumnTransformer([
    ('dummify_ecg', make_pipeline(cat_imputer, enc1), ['HomePlanet']),
    ('dummify_cp', make_pipeline(cat_imputer, enc1), ['CryoSleep']),
    ("dummify_dest", make_pipeline(cat_imputer, enc1), ['Destination']),
    ("dummify_vip", make_pipeline(cat_imputer, enc1), ['VIP']),
    ('nums', make_pipeline(num_imputer, StandardScaler()), 
     ['Age', "RoomService", "FoodCourt", "ShoppingMall","Spa", "VRDeck"])
], remainder='drop')

# Pipeline with KNN Classifier

# Final KNN pipeline
log = Pipeline([
    ('dummify_everything', ct),
    ('KNN', LogisticRegression())
])

fit = log.fit(X_train, y_train)
cross_val_score(log,X,y, scoring = 'accuracy').mean()

0.7854600775150196

In [ ]:
ct.fit

In [23]:
fit.named_steps['KNN'].coef_

array([[-0.92198576,  1.29304252, -0.34135123, -0.61942868,  0.6491342 ,
         0.34690684, -0.15657409, -0.16062722,  0.19963269, -0.16992717,
        -0.13820891, -0.93071411,  0.79433622,  0.27767192, -2.15829643,
        -1.88712399]])

In [8]:
test_preds = fit.predict(test)

In [9]:
len(test_preds)

4277

In [10]:
test_new = test.copy()

In [11]:
test_new['Transported'] = test_preds

In [12]:
submission = test_new[['PassengerId', 'Transported']]
submission.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False


In [96]:
#submission.to_csv("first_submission.csv", index=False)